# Building Machine Learning Classifiers: Random Forest on a holdout test set

### Read in & clean text

In [1]:
import nltk
import pandas as pd
import re
from sklearn.feature_extraction.text import TfidfVectorizer
import string

stopwords = nltk.corpus.stopwords.words('english')
ps = nltk.PorterStemmer()

data = pd.read_csv("SMSSpamCollection.tsv", sep='\t')
data.columns = ['label', 'body_text']

def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return round(count/(len(text) - text.count(" ")), 3)*100

data['body_len'] = data['body_text'].apply(lambda x: len(x) - x.count(" "))
data['punct%'] = data['body_text'].apply(lambda x: count_punct(x))

def clean_text(text):
    text = "".join([word.lower() for word in text if word not in string.punctuation])
    tokens = re.split('\W+', text)
    text = [ps.stem(word) for word in tokens if word not in stopwords]
    return text

tfidf_vect = TfidfVectorizer(analyzer=clean_text)
X_tfidf = tfidf_vect.fit_transform(data['body_text'])

X_features = pd.concat([data['body_len'], data['punct%'], pd.DataFrame(X_tfidf.toarray())], axis=1)
X_features.head()

,body_len,punct%,0,1,2,3,4,5,6,7,...,8094,8095,8096,8097,8098,8099,8100,8101,8102,8103
0,128,4.7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,49,4.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,62,3.2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,28,7.1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,135,4.4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Explore RandomForestClassifier through Holdout Set

In [3]:
from sklearn.metrics import precision_recall_fscore_support as score
from sklearn.model_selection import train_test_split

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X_features, data.label, test_size=0.2)

In [5]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=50, max_depth=20, n_jobs=-1)
rf_model = rf.fit(X_train, y_train)


In [7]:
sorted(zip(rf_model.feature_importances_, X_train.columns), reverse=True)

[(0.05307639759594515, 'body_len'),
 (0.04041254940634334, 1803),
 (0.03557093708676147, 5724),
 (0.03426955980041606, 2031),
 (0.02816931166148578, 4796),
 (0.02550908780698834, 7350),
 (0.02117477833776388, 7218),
 (0.01919566999581178, 1361),
 (0.015130830722337927, 3134),
 (0.013771065853478832, 6285),
 (0.013749387362953787, 7027),
 (0.013353316581608715, 5078),
 (0.01272444320387889, 6746),
 (0.012071974979374995, 'punct%'),
 (0.011775050082325626, 2299),
 (0.01165876113647974, 5988),
 (0.01162100040192995, 392),
 (0.010808923208216574, 295),
 (0.009706872720627984, 5917),
 (0.009549150251158288, 4378),
 (0.009207259366107173, 1881),
 (0.008830182194677206, 7782),
 (0.008430686660129263, 7543),
 (0.008401274476167576, 436),
 (0.008293355168788145, 397),
 (0.008067953691692553, 7461),
 (0.0077702994433137265, 3443),
 (0.007545161543293896, 354),
 (0.007493813612448831, 2171),
 (0.007298744774541253, 611),
 (0.007216761270081139, 6971),
 (0.007163922305763585, 4295),
 (0.0067773087

In [10]:
y_pred = rf_model.predict(X_test)
precision, recall, fscore, support = score(y_test, y_pred, pos_label='spam', average='binary')

In [12]:
print('Precision: ', round(precision, 3))
print('Recall: ', round(recall, 3))
print('Accuracy: ', round((y_pred == y_test).sum() / len(y_pred), 3))
print('f-Score: ', round(fscore, 3))
print('Support: ', support)

Precision:  1.0
Recall:  0.614
Accuracy:  0.952
f-Score:  0.761
Support:  None
